In [ ]:
import numpy as np
from driver import io_shape_dict
from driver_base import FINNExampleOverlay

In [2]:
bsize = 100
test_data = np.load('data.npy').astype('int8')
labels = np.load('labels.npy')
bitfile = "../bitfile/finn-accel.bit"
platform = "zynq-iodma"

In [3]:
test_data.shape

(100, 1000, 1, 62)

In [4]:
driver = FINNExampleOverlay(
    bitfile_name=bitfile,
    platform=platform,
    io_shape_dict=io_shape_dict,
    batch_size=bsize,
    runtime_weight_dir="runtime_weights/",
)

In [5]:
obuf_packed = np.empty_like(driver.obuf_packed_device[0])

In [6]:
%%time
ibuf_folded = driver.fold_input(test_data)
ibuf_packed = driver.pack_input(ibuf_folded)
driver.copy_input_data_to_device(ibuf_packed)
driver.execute_on_buffers()
driver.copy_output_data_from_device(obuf_packed)
obuf_folded = driver.unpack_output(obuf_packed)
obuf_normal = driver.unfold_output(obuf_folded)

CPU times: user 692 ms, sys: 11 ms, total: 703 ms
Wall time: 702 ms


In [9]:
pred_classes = np.argmax(obuf_normal, axis=-1).reshape(-1)

In [10]:
# Compute accuracy
accuracy = np.mean(pred_classes == labels)
print("Accuracy:", accuracy)

Accuracy: 0.71
